In [ ]:
pip install nltk scikit learn streamlit

In [ ]:
pip install nltk scikit-learn streamlit

In [ ]:
import nltk
import re
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

In [ ]:
intents = [
    {
        'tag': 'greeting',
        'patterns': ['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses': ['Hi there', 'Hello', 'Hey', 'Nothing much', "I'm fine, thank you"]
    },
    {
        'tag': 'goodbye',
        'patterns': ['Bye', 'See you later', 'Goodbye', 'Take care'],
        'responses': ['Goodbye', 'See you later', 'Take care']
    },
    {
        "tag": "eco_tips",
        "patterns": ["How can I reduce my carbon footprint?", "Give me eco-friendly tips", "Ways to save energy"],
        "responses": ["You can reduce your carbon footprint by using renewable energy, recycling, and reducing waste.", "Try using energy-efficient appliances and switching to public transportation."]
    },
    {
        "tag": "green_certifications",
        "patterns": ["What are green certifications?", "Tell me about ISO 14001"],
        "responses": ["Green certifications include ISO 14001 for environmental management and LEED for sustainable buildings.", "ISO 14001 is an international standard for environmental management systems."]
    },
    {
        "tag": "leed_certification",
        "patterns": ["What is LEED certification?", "Tell me about LEED certification", "Explain LEED certification"],
        "responses": ["LEED (Leadership in Energy and Environmental Design) is a globally recognized green building certification system developed by the U.S. Green Building Council. It provides a framework for designing, constructing, and operating high-performance, sustainable buildings. For more information, visit https://www.usgbc.org/leed"]
    },
    {
        "tag": "business_assessment",
        "patterns": ["I want to assess my business", "Start sustainability assessment"],
        "responses": [
            "Let's start your business sustainability assessment. Please answer the following questions:",
            "1. What is the name of your business?",
            "2. What industry does your business operate in?",
            "3. How many employees work in your company?",
            "4. What is your primary source of energy? (Renewable, Fossil fuels, Mixed)",
            "5. Do you use a waste recycling system? (Yes/No)",
            "6. What percentage of your waste is recycled? (e.g., 20, 50, 80)",
            "7. Do you implement water conservation measures? (Yes/No)",
            "8. What water-saving technologies or policies do you use?",
            "9. Do you track your carbon emissions? (Yes/No)",
            "10. How do you measure and offset your carbon footprint?",
            "11. Do you hold any green certifications? (Yes/No)",
            "12. Are you working towards any sustainability goals? (e.g., Net Zero, Carbon Neutral)"
        ]
    },
    {
        "tag": "report_generation",
        "patterns": ["Can I get a sustainability report?", "Generate my progress report"],
        "responses": ["Yes, you can generate a detailed sustainability report showing your improvements over time.", "Our system provides monthly/quarterly sustainability reports based on your actions."]
    }
]


In [ ]:

# Train the classifier using scikit-learn

vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

training_patterns = []
training_tags = []
for intent in intents:
    for pattern in intent['patterns']:
        training_patterns.append(pattern)
        training_tags.append(intent['tag'])

X_train = vectorizer.fit_transform(training_patterns)
y_train = training_tags
clf.fit(X_train, y_train)


# Updated classifier function with regex check

def classify_message(input_text):
    """
    Classify the user input and return a response.
    Uses regex to catch phrases like 'how are you' and returns a fixed response.
    """
    lower_text = input_text.lower().strip()
    
    # Special case for "how are you"
    if re.search(r'\bhow are you\b', lower_text):
        return "I'm fine, thank you"
    
    # Check if the input exactly matches a simple greeting
    if lower_text in ["hi", "hello", "hey"]:
        return "Hi there"
    
    # Otherwise, use the classifier
    input_vector = vectorizer.transform([input_text])
    predicted_tag = clf.predict(input_vector)[0]
    for intent in intents:
        if intent['tag'] == predicted_tag:
            return random.choice(intent['responses'])
    return "I'm sorry, I don't understand. Can you rephrase?"


# Sustainability assessment scoring functions

def calculate_score(responses):
    score = 0
    
    # Energy Source
    energy_source = responses.get('energy_source', '').lower()
    if energy_source == 'renewable':
        score += 10
    elif energy_source == 'mixed':
        score += 5
    
    # Waste Recycling System and Percentage
    waste_system = responses.get('waste_recycling_system', '').lower()
    if waste_system == 'yes':
        try:
            percent = float(responses.get('waste_recycling_percentage', 0))
        except:
            percent = 0
        if percent >= 80:
            score += 10
        elif percent >= 50:
            score += 5
    
    # Water Conservation and Water-Saving Policies
    water_conservation = responses.get('water_conservation', '').lower()
    if water_conservation == 'yes':
        score += 5
        water_policy = responses.get('water_policy', '').lower()
        keywords = ['low-flow', 'efficient', 'rainwater', 'recycle', 'recycling']
        if any(keyword in water_policy for keyword in keywords):
            score += 5
    
    # Carbon Tracking and Offset Details
    carbon_tracking = responses.get('carbon_tracking', '').lower()
    if carbon_tracking == 'yes':
        score += 5
        carbon_details = responses.get('carbon_details', '').lower()
        keywords = ['offset', 'measurement', 'tracking', 'audit', 'certified']
        if any(keyword in carbon_details for keyword in keywords):
            score += 5
    
    # Green Certifications (only if approved)
    green_certifications = responses.get('green_certifications', '').lower()
    if green_certifications == 'yes' and responses.get('certification_status', '').lower() == 'approved':
        score += 10
    
    # Sustainability Goals
    sustainability_goals = responses.get('sustainability_goals', '')
    if sustainability_goals.strip() != '':
        score += 5
    
    return score

def categorize_business(score):
    if score >= 45:
        return "Green Leader"
    elif score >= 25:
        return "Eco-Friendly Adopter"
    else:
        return "Needs Improvement"


# Command-line interactive interface

def run_chat():
    print("\n[Chat Mode] Type your message (type 'quit' to exit chat mode):")
    while True:
        msg = input("> ")
        if msg.strip().lower() == 'quit':
            break
        
        # Check if the user wants to run an assessment
        if any(phrase in msg.lower() for phrase in ["run assessment", "assess my business", "start assessment"]):
            run_assessment()
            continue  # After assessment, return to chat mode
        
        response = classify_message(msg)
        print("Bot:", response)

def run_assessment():
    print("\n[Assessment Mode] Answer the following questions (type 'quit' at any prompt to cancel):")
    responses = {}
    questions = [
       ("name", "1. What is the name of your business? "),
       ("industry", "2. What industry does your business operate in? "),
       ("employees", "3. How many employees work in your company? "),
       ("energy_source", "4. What is your primary source of energy? (Renewable, Fossil fuels, Mixed) "),
       ("waste_recycling_system", "5. Do you use a waste recycling system? (Yes/No) "),
       ("waste_recycling_percentage", "6. What percentage of your waste is recycled? (e.g., 20, 50, 80) "),
       ("water_conservation", "7. Do you implement water conservation measures? (Yes/No) "),
       ("water_policy", "8. What water-saving technologies or policies do you use? "),
       ("carbon_tracking", "9. Do you track your carbon emissions? (Yes/No) "),
       ("carbon_details", "10. How do you measure and offset your carbon footprint? "),
       ("green_certifications", "11. Do you hold any green certifications? (Yes/No) "),
       ("certification_status", "12. What is your certification status? (Approved/Pending) "),
       ("sustainability_goals", "13. Are you working towards any sustainability goals? (e.g., Net Zero, Carbon Neutral) ")
    ]
    for key, question in questions:
        answer = input(question)
        if answer.strip().lower() in ["quit", "exit"]:
            print("Assessment aborted. Returning to main menu.")
            return
        responses[key] = answer
    
    score = calculate_score(responses)
    category = categorize_business(score)
    
    print("\nAssessment Complete!")
    print("Your sustainability score is:", score)
    print("Your business category is:", category)

def main():
    while True:
        print("\nGreen Chatbot Options:")
        print("1. Chat with the bot")
        print("2. Business Sustainability Assessment")
        print("0. Exit")
        choice = input("Enter your choice: ").strip()
        if choice == '1':
            run_chat()
        elif choice == '2':
            run_assessment()
        elif choice == '0':
            print("Exiting. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == '__main__':
    main()